In [8]:
## DO NOT ERASE THIS. IMPORTANT TO CORRECTLY IMPORT MODULES
import sys

sys.path.append("../")
sys.executable

'/Users/kristina/Desktop/University/COURSE_WORK/RecSys_thesis/recsysvenv/bin/python3.12'

In [9]:
import pandas as pd

pd.set_option("display.max_rows", 102)
from tqdm.auto import tqdm

import torch
from torch.utils.data import DataLoader, TensorDataset

from sklearn.preprocessing import OrdinalEncoder, StandardScaler

from src.utils import (
    load_MovieLens,
    create_test_user,
    train_test_val_split,
    seed_everything,
    trainDatasetWithNumCatFeatures,
    split_test_df,
    add_not_watched_movies,
    create_test_user_display_df,
    train,
    predict,
)
from src.models import DeepFM
from src.metrics import reccomendation_report

## Constants

In [10]:
DATA_FOLDER = "../../data/ml-1m/"
RANDOM_STATE = 7

In [11]:
seed_everything(RANDOM_STATE)

## Data

In [12]:
df_users, df_movies, df_ratings = load_MovieLens(DATA_FOLDER)

### Creating a test_user

In [13]:
df_users, df_ratings, new_user_id = create_test_user(
    df_users, df_ratings, [6, 16, 3192, 1461, 827, 887, 593]
)
print("Test user watch list:")
display(df_ratings[df_ratings.userId == new_user_id].merge(df_movies, on="movieId"))

Test user watch list:


,userId,movieId,rating,timestamp,name,genre
0,6040,6,5,0,Sabrina (1995),Comedy|Romance
1,6040,16,5,1,Sense and Sensibility (1995),Drama|Romance
2,6040,3192,5,2,Singles (1992),Comedy|Drama|Romance
3,6040,1461,5,3,Love and Other Catastrophes (1996),Romance
4,6040,827,5,4,Emma (1996),Comedy|Drama|Romance
5,6040,887,5,5,Singin' in the Rain (1952),Musical|Romance
6,6040,593,5,6,Pretty Woman (1990),Comedy|Romance


### Train-test split
Methodology: Last user interaction is a test item. The rest is train. Validation part is 20% of test.

In [14]:
df_train, df_test, df_val = train_test_val_split(df_ratings, df_movies, RANDOM_STATE)

Enriching test:   0%|          | 0/4832 [00:00<?, ?it/s]

Enriching val:   0%|          | 0/1209 [00:00<?, ?it/s]

### Loading additional data

In [15]:
# cos_dist = computeCosineSimilarities(df_train, "userId", "movieId", df_movies["movieId"].nunique())

# with open("../../data/cos_dist.pt", "wb") as f:
#     torch.save(cos_dist, f)

with open("../../data/cos_dist.pt", "rb") as f:
    cos_dist = torch.load(f)
popularity = torch.bincount(
    torch.tensor(df_train.movieId.values), minlength=df_movies["movieId"].nunique()
)
popularity = popularity / torch.max(popularity)

### Building ordinal encoded features and normalizing continious features

In [16]:
ord_user = OrdinalEncoder()
user_cat = torch.tensor(ord_user.fit_transform(df_users[["gender", "occupation"]]))

ord_movie = OrdinalEncoder()
movie_cat = torch.tensor(ord_movie.fit_transform(df_movies[["genre"]]))

ss_user = StandardScaler()
user_num = torch.tensor(ss_user.fit_transform(df_users[["age"]]))

# features for val
user_cat_val = user_cat[df_val["userId"].values].clone().detach().to(torch.long)
user_num_val = user_num[df_val["userId"].values].clone().detach().to(torch.float)
movie_cat_val = movie_cat[df_val["movieId"].values].clone().detach().to(torch.long)
val_cat = torch.hstack((user_cat_val, movie_cat_val))

# features for test
user_cat_test = user_cat[df_test["userId"].values].clone().detach().to(torch.long)
user_num_test = user_num[df_test["userId"].values].clone().detach().to(torch.float)
movie_cat_test = movie_cat[df_test["movieId"].values].clone().detach().to(torch.long)
test_cat = torch.hstack((user_cat_test, movie_cat_test))


num_numeric_feats = 1
cat_feature_vocab = [len(i) for i in ord_user.categories_] + [
    len(i) for i in ord_movie.categories_
]

## DeepFM

In [17]:
seed_everything(RANDOM_STATE)
train_loader = DataLoader(
    trainDatasetWithNumCatFeatures(
        df_train, df_movies["movieId"].nunique(), user_cat, user_num, movie_cat
    ),
    batch_size=2048,
    shuffle=True,
)
val_loader = DataLoader(
    TensorDataset(
        torch.tensor(df_val["userId"]),
        torch.tensor(df_val["movieId"]),
        user_num_val,
        val_cat,
    ),
    batch_size=4096,
    shuffle=False,
)
test_loader = DataLoader(
    TensorDataset(
        torch.tensor(df_test["userId"]),
        torch.tensor(df_test["movieId"]),
        user_num_test,
        test_cat,
    ),
    batch_size=4096,
    shuffle=False,
)
num_users = df_users["userId"].nunique()
num_items = df_movies["movieId"].nunique()

  0%|          | 0/994175 [00:00<?, ?it/s]

In [18]:
seed_everything(RANDOM_STATE)
dF = DeepFM(
    num_users,
    num_items,
    num_numeric_feats,
    cat_feature_vocab,
    n_mlp_layers=4,
    mlp_layers_dim=32,
    mlp_kwargs={
        "activation": True,
        "dropout": True,
        "batchnorm": True,
        "dropout_rate": 0.6,
    },
)
display(dF)

DeepFM(
  (user_embedding): Embedding(6041, 5)
  (item_embedding): Embedding(3883, 5)
  (numerical_embeddings): ModuleList(
    (0): MLP(
      (block): Sequential(
        (Linear): Linear(in_features=1, out_features=5, bias=True)
      )
    )
  )
  (categorical_embeddings): ModuleList(
    (0): Embedding(2, 5)
    (1): Embedding(21, 5)
    (2): Embedding(301, 5)
  )
  (mlp): Sequential(
    (MLP_layer_0): MLP(
      (block): Sequential(
        (Linear): Linear(in_features=30, out_features=32, bias=True)
        (Activation): ReLU()
        (Dropout): Dropout(p=0.2, inplace=False)
        (BatchNorm): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (MLP_layer_1): MLP(
      (block): Sequential(
        (Linear): Linear(in_features=32, out_features=32, bias=True)
        (Activation): ReLU()
        (Dropout): Dropout(p=0.6, inplace=False)
        (BatchNorm): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=T

In [19]:
optimizer = torch.optim.Adam(dF.parameters(), lr=3e-3)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.8)
criterion = torch.nn.BCELoss()
n_epochs = 10
device = "cpu"

In [20]:
%%time
train(
    dF,
    train_loader,
    optimizer,
    scheduler,
    criterion,
    n_epochs,
    val_loader=val_loader,
    df_val=df_val,
    cos_dist=cos_dist,
    popularity=popularity,
    verbose=True,
)

Epochs:   0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/2428 [00:00<?, ?batch/s]

Epoch: 0
Train loss: 0.63687


Inference:   0%|          | 0/30 [00:00<?, ?batch/s]

{'Hit rate @ 15': tensor(0.5029),
 'NDCG @ 15': tensor(0.2335),
 'Diversity (ILD)': tensor(0.1246),
 'Novelty (EPC)': tensor(0.7349)}


...

{'Hit rate @ 15': tensor(0.6534),
 'NDCG @ 15': tensor(0.3269),
 'Diversity (ILD)': tensor(0.1290),
 'Novelty (EPC)': tensor(0.7346)}


  0%|          | 0/2428 [00:00<?, ?batch/s]

Epoch: 9
Train loss: 0.30426


Inference:   0%|          | 0/30 [00:00<?, ?batch/s]

{'Hit rate @ 15': tensor(0.6658),
 'NDCG @ 15': tensor(0.3305),
 'Diversity (ILD)': tensor(0.1285),
 'Novelty (EPC)': tensor(0.7358)}
CPU times: user 33min 16s, sys: 1h 26min 23s, total: 1h 59min 40s
Wall time: 13min 21s


In [21]:
df_test["pred"] = predict(dF, test_loader, verbose=False).numpy()
pred, target, pred_items = split_test_df(df_test, "userId", "movieId", "pred", "action")
reccomendation_report(pred, target, pred_items, cos_dist, popularity, k=15)

{'Hit rate @ 15': tensor(0.6445),
 'NDCG @ 15': tensor(0.3247),
 'Diversity (ILD)': tensor(0.1292),
 'Novelty (EPC)': tensor(0.7335)}

## Test user recommendations

In [22]:
df_test_user = add_not_watched_movies(new_user_id, df_test, df_train, df_movies)

# features for test user
user_cat_test_user = (
    user_cat[df_test_user["userId"].values].clone().detach().to(torch.long)
)
user_num_test_user = (
    user_num[df_test_user["userId"].values].clone().detach().to(torch.float)
)
movie_cat_test_user = (
    movie_cat[df_test_user["movieId"].values].clone().detach().to(torch.long)
)
test_user_cat = torch.hstack((user_cat_test_user, movie_cat_test_user))

test_user_loader = DataLoader(
    TensorDataset(
        torch.tensor(df_test_user["userId"]),
        torch.tensor(df_test_user["movieId"]),
        user_num_test_user,
        test_user_cat,
    ),
    batch_size=4096,
    shuffle=False,
)

In [23]:
df_test_user["pred"] = predict(dF, test_user_loader, verbose=True).numpy()
display(create_test_user_display_df(df_test_user, df_movies, "pred"))

Inference:   0%|          | 0/1 [00:00<?, ?batch/s]

,userId,movieId,name,genre,pred
0,6040,1211,Annie Hall (1977),Comedy|Romance,0.970746
1,6040,1227,"Graduate, The (1967)",Drama|Romance,0.963844
2,6040,2222,Edward Scissorhands (1990),Drama|Romance,0.960397
3,6040,591,Beauty and the Beast (1991),Animation|Children's|Musical,0.958382
4,6040,1078,"Crying Game, The (1992)",Drama|Romance|War,0.956313
5,6040,2728,Big (1988),Comedy|Fantasy,0.906145
6,6040,1878,West Side Story (1961),Musical|Romance,0.901637
7,6040,523,Schindler's List (1993),Drama|War,0.892324
8,6040,2018,Peter Pan (1953),Animation|Children's|Fantasy|Musical,0.874618
9,6040,582,Home Alone (1990),Children's|Comedy,0.835076
